<a href="https://colab.research.google.com/github/rafael-rosa/mack-modelos-linguagem-generativos/blob/main/movie_review_classif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PROJETO FINAL DA DISCIPLINA ***MODELOS DE LINGUAGEM E GENERATIVOS***
#### Prof. Rogério de Oliveira
Alunos: 
+ `Gildo Manzi da Silva - RA: 10329658`
+ `Rafael da Silva Rosa - RA: 10746329`
+ `Rogério Goussain Labat - RA: 10746326`

#### 🎬 Objetivo: Classificar reviews de filmes em ***"Positivo"*** ou ***"Negativo"*** 🎭
#### 📊 Origem dos dados: **IMDB / Kaggle**

## 1️⃣ SETUP

In [1]:
!pip install --upgrade pip
!pip install -r requirements.txt
!pip install -r other-requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu129


#### 1️⃣.1️⃣ IMPORTS

In [2]:
import kagglehub 
import pandas as pd
import torch
from transformers import pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate

c:\dev\python\wks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1️⃣.2️⃣ ENV VARIABLES

In [3]:
AMOUNT_OF_REVIEWS_TO_CLASSIFY = 300
NUM_TRAIN_EPOCHS = 3 # 2 ou 3 épocas
WARMUP_STEPS = 500 # Passos de aquecimento para o otimizador
WEIGHT_DECAY = 0.01 # Decaimento de peso para o otimizador
TRAIN_SAMPLE_SIZE = 4000 # Tamanho da amostra de treino
MODEL_CHECKPOINT = "distilbert-base-uncased" # modelo leve para o fine-tuning - DistilBERT - rápido de treinar e tem ótima performance.
KAGGLE_DATASET = "lakshmi25npathi/imdb-dataset-of-50k-movie-reviews" # Dataset do Kaggle

#### 1️⃣.3️⃣ DATASET - IMDB MOVIE REVIEWS TO CLASSIFY

In [4]:
# Download dataset IMDB reviews
path = kagglehub.dataset_download(KAGGLE_DATASET)

print("Path to dataset files:", path)
df = pd.read_csv(path + "/IMDB Dataset.csv")    
df.head(20)

Path to dataset files: C:\Users\rafas\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


#### 1️⃣.4️⃣ SET GPU CARD

In [5]:
!nvidia-smi
print(f"GPU available? {'✅' if torch.cuda.is_available() else '❌'}")
print(f"GPU name: {'🎮 ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else '⚠️ None'}")

# Verify if the GPU (CUDA) is available
# If it is, use "cuda:0" (the first GPU)
# If not, use "cpu"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(f"Selected device: 🖥️ {device}")

Mon Nov 17 00:10:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.00                 Driver Version: 577.00         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8              1W /   60W |    3881MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device=device)   # Use GPU if available

Device set to use cuda:0


## 2️⃣ Zero-Shot Classification 
#### 2️⃣.1️⃣ USE ***BART LARGE MNLI*** MODEL TO CLASSIFY MOVIE REVIEWS IN POSITIVE/NEGATIVE

In [7]:
reviews = df['review'][:AMOUNT_OF_REVIEWS_TO_CLASSIFY].tolist()
print(f"Classifying {len(reviews)} reviews...")

model_classification_results = []
candidate_labels = ['positive', 'negative']

def data():
    for i in reviews:
        yield i

# Batch processing using the pipeline for efficiency
for result in classifier(data(), candidate_labels, batch_size=2):
    print("classification:", result['labels'][0], "with score", result['scores'][0])
    model_classification_results.append({
        'review': result['sequence'],
        'predicted_label': result['labels'][0],
        'score': result['scores'][0]
    })

Classifying 300 reviews...
classification: positive with score 0.5315352082252502
classification: positive with score 0.9874508380889893
classification: positive with score 0.9743621349334717
classification: negative with score 0.9753219485282898
classification: positive with score 0.8379416465759277
classification: positive with score 0.9739866256713867
classification: positive with score 0.7787739038467407
classification: negative with score 0.9267167448997498
classification: negative with score 0.8084462881088257
classification: positive with score 0.9694986939430237
classification: negative with score 0.8875364065170288
classification: negative with score 0.5397161841392517
classification: negative with score 0.8973174691200256
classification: negative with score 0.9752566814422607
classification: positive with score 0.9951791167259216
classification: negative with score 0.9860522150993347
classification: negative with score 0.870980441570282
classification: negative with score 0.9

#### 2️⃣.2️⃣ ERROR PERCENTAGE

In [8]:
counter_incorrect = 0
for index, row in df.head(AMOUNT_OF_REVIEWS_TO_CLASSIFY).iterrows():
    predicted = model_classification_results[index]['predicted_label']
    actual_label = row['sentiment']
    if predicted != actual_label:
        # print(f"Review {index+1} - Predicted: {predicted}, Actual: {actual_label}")
        counter_incorrect += 1
        
error_percentage = (counter_incorrect / AMOUNT_OF_REVIEWS_TO_CLASSIFY) * 100
print(f"Total incorrect predictions: {counter_incorrect} - Error percentage: {error_percentage:.2f}%")

Total incorrect predictions: 30 - Error percentage: 10.00%


#### 2️⃣.3️⃣ METRICS - BASELINE

In [9]:
# 1. Obter os rótulos verdadeiros (ground truth)
true_labels = list(df.head(AMOUNT_OF_REVIEWS_TO_CLASSIFY)['sentiment'])

# 2. Obter os rótulos previstos
predicted_labels = [item['predicted_label'] for item in model_classification_results]

# 3. Relatório de classificação
print("--- MÉTRICAS DO MODELO ZERO-SHOT (Baseline) ---")
print(classification_report(true_labels, predicted_labels, target_names=['negative', 'positive']))

--- MÉTRICAS DO MODELO ZERO-SHOT (Baseline) ---
              precision    recall  f1-score   support

    negative       0.88      0.94      0.91       161
    positive       0.93      0.85      0.89       139

    accuracy                           0.90       300
   macro avg       0.90      0.90      0.90       300
weighted avg       0.90      0.90      0.90       300



## 3️⃣ FINE-TUNNING
#### 3️⃣.1️⃣ DATA PREPARATION

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

sample_df = df.sample(n=TRAIN_SAMPLE_SIZE, random_state=42) 

# Convert 'positive'/'negative' to 0 and 1
sample_df['label_num'] = sample_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Split the data into Train and Test
X = list(sample_df['review'])
y = list(sample_df['label_num'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize (convert text into numbers the model understands)
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=256)

In [11]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Get all 'input_ids', 'attention_mask', etc.
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add label
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, y_train)
test_dataset = IMDbDataset(test_encodings, y_test)

#### 3️⃣.2️⃣ TREINAMENTO

In [12]:
accuracy_metric = evaluate.load("accuracy")

# Used by trainer to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Load the pre-trained model (with a classification "head" on top)
# The num_labels=2 tells it to prepare to classify between 2 things (pos/neg)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)

# Uma prática melhor é avaliar o modelo ao final de cada época e salvar apenas o melhor.
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",            # Avaliar a cada época
    save_strategy="epoch",            # Salvar a cada época
    load_best_model_at_end=True       # Carregar o melhor modelo no final
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.410700,0.431061,0.813750
2,0.299200,0.333066,0.880000
3,0.187100,0.313103,0.897500


TrainOutput(global_step=600, training_loss=0.35445082326730093, metrics={'train_runtime': 169.738, 'train_samples_per_second': 56.558, 'train_steps_per_second': 3.535, 'total_flos': 635843513548800.0, 'train_loss': 0.35445082326730093, 'epoch': 3.0})

### 3️⃣.3️⃣ METRICS - AFTER FINE-TUNNING

In [13]:
# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Predictions come out as "logits", we need the final label (0 or 1)
predicted_labels_tuned = np.argmax(predictions.predictions, axis=1)

# Get the true labels from the test set
true_labels_tuned = y_test

# Generate the classification report
print("--- FINE-TUNED MODEL METRICS ---")
print(classification_report(true_labels_tuned, predicted_labels_tuned, target_names=['negative', 'positive']))

--- FINE-TUNED MODEL METRICS ---
              precision    recall  f1-score   support

    negative       0.88      0.91      0.90       392
    positive       0.91      0.88      0.90       408

    accuracy                           0.90       800
   macro avg       0.90      0.90      0.90       800
weighted avg       0.90      0.90      0.90       800



## 4️⃣ CONCLUSÃO

O objetivo deste projeto foi comparar a performance de um modelo de classificação Zero-Shot (Baseline) contra um modelo Fine-Tuned (Especialista) na tarefa de análise de sentimento.

### Resultados:

| MODELO | ESTRATÉGIA | MODELO BASE | ACURÁCIA | F1-SCORE (WEIGHTED) |
| -------- | -------- | ----------- | -------- | ------------------- |
| Baseline | Zero-Shot | ***facebook/bart-large-mnli*** | 90% | 0.9 |
| Desafiante | Fine-Tuning | ***distilbert-base-uncased*** | 90% | 0.90 |

### Análise:

Surpreendentemente, o modelo Fine-Tuned (DistilBERT com 90% de acurácia) não superou o modelo Zero-Shot (BART-Large, também com 90%).
Isso sugere que a tarefa de classificação de sentimento binário (positivo/negativo) é uma tarefa onde os modelos NLI de grande escala, como o BART-Large (406M de parâmetros), já possuem uma capacidade generalista extremamente alta.
Mesmo treinando um modelo especialista (DistilBERT, 66M de parâmetros) com 4000 amostras, ele apenas conseguiu igualar a performance do modelo maior, que não recebeu nenhum treinamento específico para esta tarefa. Isso demonstra o poder dos modernos modelos de fundação.

### Observação CPU x GPU

O uso de GPU para execução do modelo **BART-Large** e treino do modelo **DistilBERT** apresentou uma redução no tempo de execução de 15 vezes frente ao uso de CPU para as mesmas tarefas. Isto deixa clara a importância do uso deste tipo de hardware no âmbito da NLI.